# Exemplu de sistem expert cu constrângeri

Un sistem expert bazat pe constrângeri este un tip de sistem expert care utilizează un set de reguli și relații logice (constrângeri) pentru a rezolva probleme complexe prin eliminarea soluțiilor incompatibile.

Acest sistem funcționează prin definirea variabilelor, a domeniilor posibile pentru acestea și a constrângerilor care descriu relațiile dintre variabile, apoi utilizează metode de inferență pentru a găsi combinațiile valide ce respectă toate constrângerile impuse.

Acest tip de sistem este frecvent utilizat în planificare, alocarea resurselor, configurarea produselor sau rezolvarea problemelor din domenii tehnice și inginerești.

## Exemplu

Sistemul generează un orar, pornind de la o serie de constrângeri predefinite

Avem 10 persoane, 3 sali, două intervale ale zilei, fiecare cu sloturi de 2 ore. Cele 10 persoane anunță o serie de constrângeri:

Persoana| Conditii
----|---------------------------------
P1 | dimineata doar în camerele 2 si 3
P2 | dimineata dar nu in aceeasi camera cu P1
P3 | dupa-amiază și camera 3
P4 | dupa-amiază și camerele 2 si 3
P5 | dimineața, camera 1 si în același timp cu P1
P6 | dupa amiaza, orice camera
P7 | oricand dar doar camera 2
P8 | dupa amiaza in camera 1
P9 | dimineata si NU in camera 3
P10 | oricând si în orice cameră

In [1]:
import itertools

### Definirea structurilor de date

Definim camerele. Acestea participa de asemenea in constrangerile definite ulterior

In [2]:
camere = [1, 2, 3]

Definim sloturile de timp

 - M - morning, dimineata
 - A - afternoon, dupa amiaza

 Aceste sloruti de timp sunt ele însele constrângeri - programarea ședințelor nu poate avea loc în afara acestor intervale de timp

In [3]:
sloturi = {
    'M': ['8-10', '10-12'],
    'A': ['12-14', '14-16', '16-18']
}

definim perioadele

In [4]:
perioade = ['M', 'A']

Inițializăm lista de persoane. Descrise aici simplu, ca P1, P2, ... etc. acestea pot avea și nume, prenume fara probleme.

In [5]:
persoane = [f'P{i}' for i in range(1, 11)]

In [ ]:
print(persoane)

['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']


Implementăm conditiile fiecărei persoane. Aici se pun doar relațiile dintre persoane si camere, relatiile dintre persoane se implementeaza ulterior. Camerele sunt reprezentate prin numarul intreg corespunzator fiecareia (1,2,3)

In [9]:
def expanded_possible_slots(persoana):
    if persoana == 'P1':
        return [(slot, r) for slot in sloturi['M'] for r in [2, 3]]
    elif persoana == 'P2':
        return [(slot, r) for slot in sloturi['M'] for r in camere]
    elif persoana == 'P3':
        return [(slot, 3) for slot in sloturi['A']]
    elif persoana == 'P4':
        return [(slot, r) for slot in sloturi['A'] for r in [2, 3]]
    elif persoana == 'P5':
        return [(slot, 1) for slot in sloturi['M']]
    elif persoana == 'P6':
        return [(slot, r) for slot in sloturi['A'] for r in camere]
    elif persoana == 'P7':
        return [(slot, 2) for t in perioade for slot in sloturi[t]]
    elif persoana == 'P8':
        return [(slot, 1) for slot in sloturi['A']]
    elif persoana == 'P9':
        return [(slot, r) for slot in sloturi['M'] for r in camere if r != 3]
    elif persoana == 'P10':
        return [(slot, r) for t in perioade for slot in sloturi[t] for r in camere]
    else:
        return [(slot, r) for t in perioade for slot in sloturi[t] for r in camere]


Implementăm regulile interpersonale, separat de cele de poziționare în sloturile orare.

In [8]:
def is_valid(schedule):
    slots = {p: s for p, s in schedule}
    if slots['P1'][0] == slots['P2'][0]:  # P2 nu poate fi in acelasi timp in cladire cu P1
        return False
    if slots['P1'][0] != slots['P5'][0]:  # P5 trebuie sa fie prezenta in acelasi timp cu P1
        return False
    return True

Construim o lista a tuturor sloturilor eligibile pentru fiecare persoană

In [10]:
cerinte = {p: expanded_possible_slots(p) for p in persoane}

Putem inspecta structura de date obținută:

In [11]:
for persoana, conditii in cerinte.items():
  print(persoana, conditii)

P1 [('8-10', 2), ('8-10', 3), ('10-12', 2), ('10-12', 3)]
P2 [('8-10', 1), ('8-10', 2), ('8-10', 3), ('10-12', 1), ('10-12', 2), ('10-12', 3)]
P3 [('12-14', 3), ('14-16', 3), ('16-18', 3)]
P4 [('12-14', 2), ('12-14', 3), ('14-16', 2), ('14-16', 3), ('16-18', 2), ('16-18', 3)]
P5 [('8-10', 1), ('10-12', 1)]
P6 [('12-14', 1), ('12-14', 2), ('12-14', 3), ('14-16', 1), ('14-16', 2), ('14-16', 3), ('16-18', 1), ('16-18', 2), ('16-18', 3)]
P7 [('8-10', 2), ('10-12', 2), ('12-14', 2), ('14-16', 2), ('16-18', 2)]
P8 [('12-14', 1), ('14-16', 1), ('16-18', 1)]
P9 [('8-10', 1), ('8-10', 2), ('10-12', 1), ('10-12', 2)]
P10 [('8-10', 1), ('8-10', 2), ('8-10', 3), ('10-12', 1), ('10-12', 2), ('10-12', 3), ('12-14', 1), ('12-14', 2), ('12-14', 3), ('14-16', 1), ('14-16', 2), ('14-16', 3), ('16-18', 1), ('16-18', 2), ('16-18', 3)]


Următoarea funcție generează toate combinațiile posibile de camere, sloturi de timp intr-o lista de 10 tupluri, fiecare tuplu corespunzând unei persoane.
Aceasta lista va contine un numar foarte mare de combinatii.

Person | sloturi posibile
-------|----------------
P1     | 4
P2     | 6
P3     | 3
P4     | 6
P5     | 2
P6     | 9
P7     | 5
P8     | 3
P9     | 4
P10    | 15

Cantitatea totală de combinatii este 4 x 6 x 3 x 6 x 2 x 9 x 5 x 3 x 4 x 15 = 6998400

In [12]:
def manual_product(cerinte, persoane):
    # Incepem cu o lista care contine un tuplu gol (care va pastra prima combinatie)
    result = [()]

    # Cicleaza prin toate persoanele din lista
    for p in persoane:
        # Creaza o noua lista care va pastra toate combinatiile pentru aceasta persoana
        new_result = []

        # Loop through the existing combinations in 'result'

        for combination in result:
            # Trecem prin fiecare slot si camera posibile pentru persoana curenta
            for slot in cerinte[p]:
                # Adaugam o noua combinatie (slot, camera)
                new_combination = combination + (slot,)
                new_result.append(new_combination)

        # Dupa procesarea tuturor combinatiilor pentru persoana curenta, atribuim noul rezultat variabilei principale
        result = new_result

    return result

Un exemplu simplificat de produs cartezian se vede in urmatoarele celule

In [13]:
minicerinte = {
    'Ana': [1, 2],
    'Ion': [3, 4]
}
minipersoane = ['Ana', 'Ion']

In [14]:
cartezian = manual_product(minicerinte, minipersoane)
print(cartezian)

[(1, 3), (1, 4), (2, 3), (2, 4)]


### Generarea tuturor combinatiilor posibile

In continuare, generam toate combinatiile posibile intre sloturi orare si camere, in grupuri de cate 10 (primul grup corespunde primei persoane... etc.)


In [15]:
all_combinations = manual_product(cerinte, persoane)


Pentru a vedea rezultatul (partial) al produsului cartezian

In [16]:
for item in all_combinations[:10]:
    print(item)

(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 1))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 2))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 3))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('10-12', 1))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('10-12', 2))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('10-12', 3))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('12-14', 1))
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 

In [17]:
valid_schedule = None


Acest cod parcurge combinațiile posibile de alocări pentru persoane, unde fiecare combinație (combo) conține perechi de tipul (interval orar, număr cameră) pentru fiecare persoană. Fiecare combinatie (combo) este asociat cu lista de persoane prin funcția zip, rezultând o lista (schedule) care indică cine unde și când este programat. Dacă există duplicate în combo, adică două persoane au fost alocate în aceeași cameră în același interval, atunci combinația este respinsă (se continuă cu următoarea). În cazul în care alocarea este validă din perspectiva regulilor definite în funcția is_valid, atunci această planificare este salvată și căutarea se oprește.

zip creaza un iterator de tupluri intre doua liste. Un exemplu simplu, in afara codului principal


In [18]:
fructe = ['mar', 'banana', 'portocala']
culori = ['rosu', 'galben', 'portocaliu']

for fruct, culoare in zip(fructe, culori):
    print(f"{fruct} are culoarea {culoare}")

mar are culoarea rosu
banana are culoarea galben
portocala are culoarea portocaliu


În continuare, codul va parcurge toate combinatiile generate

In [19]:
for combo in all_combinations[:5]:
    print(combo)
    schedule = list(zip(persoane, combo))

    print(schedule)

(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 1))
[('P1', ('8-10', 2)), ('P2', ('8-10', 1)), ('P3', ('12-14', 3)), ('P4', ('12-14', 2)), ('P5', ('8-10', 1)), ('P6', ('12-14', 1)), ('P7', ('8-10', 2)), ('P8', ('12-14', 1)), ('P9', ('8-10', 1)), ('P10', ('8-10', 1))]
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 2))
[('P1', ('8-10', 2)), ('P2', ('8-10', 1)), ('P3', ('12-14', 3)), ('P4', ('12-14', 2)), ('P5', ('8-10', 1)), ('P6', ('12-14', 1)), ('P7', ('8-10', 2)), ('P8', ('12-14', 1)), ('P9', ('8-10', 1)), ('P10', ('8-10', 2))]
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 3))
[('P1', ('8-10', 2)), ('P2', ('8-10', 1)), ('P3', ('12-14', 3)), ('P4', ('12-14', 2)), ('P5', ('8-10', 1)), ('P6', ('12-14', 1)), ('P7', ('8-10', 2)), ('P8', ('12-

### Selectia combinatiilor corecte

În continuare verificam care dintre combinatiile generate poate fi corecta. Pentru a fi corecta trebuie sa eliminam toate combinatiile care au perechi slot-camera care apar de mai multe ori si sa verificam care dintre ele nu incalca regulile suplimentare de asociere. Pentru a observa cum arata combinatiile care au dubluri putem inspecta o mica parte din acestea, in codul urmator:

In [20]:
doubles = []
for combo in all_combinations:
    schedule = list(zip(persoane, combo))
    if len(combo) != len(set(combo)):
      doubles.append(combo)

for el in doubles[:5]:
  print(el)
  print(set(el))

(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 1))
{('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 2), ('12-14', 1)}
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 2))
{('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 2), ('12-14', 1)}
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('8-10', 3))
{('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 3), ('8-10', 2), ('12-14', 1)}
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('10-12', 1))
{('10-12', 1), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 2), ('12-14', 1)}
(('8-10', 2), ('8-10', 1), ('12-14', 3), ('12-14', 2), ('8-10', 1), ('12-14', 1), ('8-10', 2), ('12-14', 1), ('8-10', 1), ('10-12', 2))
{('8-10', 1), ('12-1

In [25]:
valid_schedules = []
for combo in all_combinations:
    schedule = list(zip(persoane, combo))
    if len(combo) != len(set(combo)):
        continue  # conflict: camera a fost deja rezervata
    if is_valid(schedule):
        valid_schedule = schedule
        valid_schedules.append(schedule)
        break


In [26]:
# Imprima rezultatul
if valid_schedule:
    valid_schedule.sort()
    print(f"{'persoana':<6} {'Time':<7} {'Camera'}")
    print('-' * 20)
    for persoana, (slot, room) in valid_schedule:
        print(f"{persoana:<6} {slot:<7} {room}")
else:
    print("Nu am gasit nici o solutie.")

persoana Time    Camera
--------------------
P1     8-10    2
P10    8-10    3
P2     10-12   1
P3     12-14   3
P4     12-14   2
P5     8-10    1
P6     12-14   1
P7     14-16   2
P8     14-16   1
P9     10-12   2


### Inspectia altor solutii

Aceasta metoda va returna doar o combinatie. In realitate insa, aproape de fiecare data exista mai multe combinatii valide. Pentru a vedea numarul de combinatii valide putem altera putin codul astfel incat sa nu se opreasca la identificarea primului rezultat valid.

In [31]:
all_valid_schedules = []
for combo in all_combinations:
    schedule = list(zip(persoane, combo))
    if len(combo) != len(set(combo)):
        continue  # conflict: camera a fost deja rezervata
    if is_valid(schedule):
        all_valid_schedule = schedule
        all_valid_schedules.append(schedule)
        # break


print(len(valid_schedules))

97200


Observam ca exista 97200 posibile solutii. In codul urmator putem vedea cum arata solutia numarul 123

In [32]:
valid_schedule = all_valid_schedules[128]
if valid_schedule:
    valid_schedule.sort()
    print(f"{'persoana':<6} {'Time':<7} {'Camera'}")
    print('-' * 20)
    for persoana, (slot, room) in valid_schedule:
        print(f"{persoana:<6} {slot:<7} {room}")
else:
    print("Nu am gasit nici o solutie.")

persoana Time    Camera
--------------------
P1     8-10    2
P10    14-16   1
P2     10-12   1
P3     12-14   3
P4     12-14   2
P5     8-10    1
P6     16-18   2
P7     14-16   2
P8     12-14   1
P9     10-12   2
